In [190]:
!pip install pandas
!pip install bokeh
!pip install pandas_bokeh

In [191]:
import pandas as pd
import pandas_bokeh
import bokeh

In [192]:
# Embedding plots in Jupyter Notebook
pandas_bokeh.output_notebook()

Loading BokehJS ...

# Data overview

In [193]:
data_url = 'https://raw.githubusercontent.com/isabelyb/hospital_charges_USA/main/hospital-charges.csv'
df = pd.read_csv(data_url)
df.head()

,DRG Definition,Provider Id,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Hospital Referral Region Description,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,AL - Dothan,91,$32963.07,$5777.24,$4763.73
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,AL - Birmingham,14,$15131.85,$5787.57,$4976.71
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,AL - Birmingham,24,$37560.37,$5434.95,$4453.79
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10011,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,AL - Birmingham,25,$13998.28,$5417.56,$4129.16
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10016,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,ALABASTER,AL,35007,AL - Birmingham,18,$31633.27,$5658.33,$4851.44


In [194]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163065 entries, 0 to 163064
Data columns (total 12 columns):
 #   Column                                Non-Null Count   Dtype 
---  ------                                --------------   ----- 
 0   DRG Definition                        163065 non-null  object
 1   Provider Id                           163065 non-null  int64 
 2   Provider Name                         163065 non-null  object
 3   Provider Street Address               163065 non-null  object
 4   Provider City                         163065 non-null  object
 5   Provider State                        163065 non-null  object
 6   Provider Zip Code                     163065 non-null  int64 
 7   Hospital Referral Region Description  163065 non-null  object
 8    Total Discharges                     163065 non-null  int64 
 9    Average Covered Charges              163065 non-null  object
 10   Average Total Payments               163065 non-null  object
 11  Average Medic

# Format names and data

In [195]:
# To remove spaces and rename columns
df.columns = df.columns.str.strip()
df.rename(columns={'Average Covered Charges': 'Avg Covered Charges [$]',
                    'Average Total Payments' : 'Avg Total Payments [$]', 
                    'Average Medicare Payments': 'Avg Medicare Payments [$]'
                    }, inplace=True)

In [196]:
# To remove $  symbol and convert into float
df[['Avg Covered Charges [$]', 'Avg Total Payments [$]', 'Avg Medicare Payments [$]']] =  df[['Avg Covered Charges [$]', 
                'Avg Total Payments [$]', 'Avg Medicare Payments [$]']].replace('\$','',regex=True).astype(float)

In [197]:
# To add 'Avg. Cost/attended patient [$]'
df['Avg. Cost/attended patient [$]'] = df['Avg Total Payments [$]']/df['Total Discharges']

In [198]:
df.sample(2)

,DRG Definition,Provider Id,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Hospital Referral Region Description,Total Discharges,Avg Covered Charges [$],Avg Total Payments [$],Avg Medicare Payments [$],Avg. Cost/attended patient [$]
144507,690 - KIDNEY & URINARY TRACT INFECTIONS W/O MCC,500030,ST JOSEPH MEDICAL CENTER,2901 SQUALICUM PARKWAY,BELLINGHAM,WA,98225,WA - Seattle,123,17688.49,5235.00,4365.64,42.560976
3818,064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...,470003,FLETCHER ALLEN HOSPITAL OF VERMONT,111 COLCHESTER AVE,BURLINGTON,VT,5401,VT - Burlington,70,27167.42,17682.87,16408.02,252.612429


# Answer questions

### 1. Which are the most expensive states and the cheapest ones?
The best variable to compare is **Average Total Payments** instead of **Average Covered Charges** because the last one is only the ticket price, not the real amount paid.

But...

In [199]:
# To answer if exist some correlation between Average Covered Charges, Average Total Payments and Average Medicare Payments
df[['Avg Covered Charges [$]', 'Avg Total Payments [$]', 'Avg Medicare Payments [$]']].corr(method ='pearson')

,Avg Covered Charges [$],Avg Total Payments [$],Avg Medicare Payments [$]
Avg Covered Charges [$],1.000000,0.774112,0.768927
Avg Total Payments [$],0.774112,1.000000,0.989362
Avg Medicare Payments [$],0.768927,0.989362,1.000000


In [200]:
df.drop(df[['Provider Id', 'Provider Zip Code', 'Avg Covered Charges [$]']], axis=1, inplace=True)

In [201]:
df.plot_bokeh.hist()

Figure(id='4413', ...)

In [202]:
df_by_state = df.groupby(df['Provider State']).sum()
df_by_state.drop(df[['Total Discharges']], axis=1, inplace=True)

In [203]:
df_by_state = df_by_state.astype(int)
df_by_state.head(4)

,Avg Total Payments [$],Avg Medicare Payments [$],Avg. Cost/attended patient [$]
Provider State,,,
AK,3366222,2993521,176355
AL,27510523,23329455,1194685
AR,16575787,14303062,695235
AZ,28950559,25162119,1332213


In [204]:
sorted_df_by_state = df_by_state.sort_values(by=['Avg Total Payments [$]'])

In [205]:
by_states = sorted_df_by_state.plot_bokeh.bar(figsize=(1200,400), 
                                            title='Hospital Charges in America by States', 
                                            show_figure=True, legend = 'top_left', disable_scientific_axes='y')

####  By Pacient by State

In [206]:
df_by_patient = df.groupby(df['Provider State']).median()

In [207]:
sorted_df_by_patient = df_by_patient.sort_values(by=['Avg. Cost/attended patient [$]'])

In [208]:
by_patient = sorted_df_by_patient.plot_bokeh.bar(figsize=(1200,400), 
                                y='Avg. Cost/attended patient [$]',
                                title='Hospital Charges in America by Attended Patient', 
                                show_figure=True, legend = 'top_left')

#### More expensive States by DRG

In [209]:
df_by_drg = df.groupby(['DRG Definition', 'Provider State']).median()
df_by_drg = df_by_drg.sort_values(['DRG Definition','Avg. Cost/attended patient [$]'], ascending=[1,0])
df_by_drg

Total Discharges  \
DRG Definition                           Provider State                     
039 - EXTRACRANIAL PROCEDURES W/O CC/MCC DC                          14.0   
                                         HI                          24.0   
                                         WY                          22.0   
                                         CA                          19.0   
                                         UT                          18.0   
...                                                                   ...   
948 - SIGNS & SYMPTOMS W/O MCC           NV                          29.0   
                                         NH                          25.5   
                                         MD                          38.0   
                                         MI                          31.0   
                                         WV                          26.0   

                                                         Avg Total Payments [$]  \
DRG Definition                           Provider State                           
039 - EXTRACRANIAL PROCEDURES W/O CC/MCC DC                            9479.570   
                                         HI                           10982.040   
                                         WY                            8961.380   
                                         CA                            7863.140   
                                         UT                            6861.240   
...                                                                         ...   
948 - SIGNS & SYMPTOMS W/O MCC           NV                            4583.360   
                                         NH                            4222.220   
                                         MD                            6732.925   
                                         MI                            4526.850   
                                         WV                            3790.255   

                                                         Avg Medicare Payments [$]  \
DRG Definition                           Provider State                              
039 - EXTRACRANIAL PROCEDURES W/O CC/MCC DC                               8680.140   
                                         HI                               6846.870   
                                         WY                               6866.105   
                                         CA                               6639.330   
                                         UT                               4799.190   
...                                                                            ...   
948 - SIGNS & SYMPTOMS W/O MCC           NV                               3672.965   
                                         NH                               3440.250   
                                         MD                               5916.040   
                                         MI                               3750.710   
                                         WV                               3168.590   

                                                         Avg. Cost/attended patient [$]  
DRG Definition                           Provider State                                  
039 - EXTRACRANIAL PROCEDURES W/O CC/MCC DC                                  677.112143  
                                         HI                                  457.585000  
                                         WY                                  453.055357  
                                         CA                                  419.425625  
                                         UT                                  413.151455  
...                                                                                 ...  
948 - SIGNS & SYMPTOMS W/O MCC           NV                                  165.056543  
                                         NH   

In [210]:
sorted_df_by_drg = df_by_drg.groupby(level=0).head(1)
sorted_df_by_drg.head(3)

,,Total Discharges,Avg Total Payments [$],Avg Medicare Payments [$],Avg. Cost/attended patient [$]
DRG Definition,Provider State,,,,
039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,DC,14.0,9479.57,8680.140,677.112143
057 - DEGENERATIVE NERVOUS SYSTEM DISORDERS W/O MCC,AK,11.0,8799.00,8081.540,799.909091
064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W MCC,AK,22.5,21608.01,20555.255,1417.867941


In [211]:
drg_by_states = sorted_df_by_drg.index.get_level_values('Provider State')
drg_by_states

Index(['DC', 'AK', 'AK', 'AK', 'HI', 'AK', 'UT', 'HI', 'DC', 'HI', 'AK', 'DC',
       'WY', 'AK', 'AK', 'HI', 'AK', 'HI', 'AK', 'WY', 'CA', 'OR', 'AK', 'AK',
       'HI', 'VT', 'WY', 'AK', 'HI', 'WY', 'AK', 'AK', 'WY', 'DC', 'HI', 'UT',
       'VT', 'HI', 'AK', 'AK', 'AK', 'WY', 'NM', 'AK', 'HI', 'WY', 'AK', 'AK',
       'WY', 'AK', 'AK', 'VT', 'AK', 'AK', 'WY', 'AK', 'AK', 'AK', 'AK', 'VT',
       'HI', 'WY', 'HI', 'RI', 'AK', 'DC', 'HI', 'DC', 'HI', 'WY', 'DC', 'AK',
       'AK', 'HI', 'HI', 'HI', 'VT', 'HI', 'AK', 'AK', 'WY', 'HI', 'AK', 'AK',
       'VT', 'HI', 'AK', 'NV', 'HI', 'AK', 'AK', 'DC', 'AK', 'WY', 'AK', 'CT',
       'AK', 'SD', 'VT', 'AK'],
      dtype='object', name='Provider State')

In [212]:
from collections import Counter
drg_max_states_counter = Counter(drg_by_states)
drg_max_states_dict = dict(drg_max_states_counter)
drg_max_states_df = pd.DataFrame(drg_max_states_dict.items())
drg_max_states_df.rename(columns={0: 'Provider State',1:'DRG_qty'}, inplace=True)
drg_max_states_df = drg_max_states_df.set_index('Provider State')

In [213]:
sorted_drg_max_states_df = drg_max_states_df.sort_values(by='DRG_qty')
sorted_drg_max_states_df.plot_bokeh.bar(show_figure=True, legend = 'top_left', title='States with more expensive DRG')

Figure(id='5029', ...)

#### More sheapest States by DRG

In [214]:
df_by_drg_min = df_by_drg.sort_values(['DRG Definition','Avg. Cost/attended patient [$]'], ascending=[1,1])

In [215]:
sorted_df_by_drg_min = df_by_drg_min.groupby(level=0).head(1)
sorted_df_by_drg_min.head(3)

,,Total Discharges,Avg Total Payments [$],Avg Medicare Payments [$],Avg. Cost/attended patient [$]
DRG Definition,Provider State,,,,
039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,ME,75.0,7225.66,5470.510,96.342133
057 - DEGENERATIVE NERVOUS SYSTEM DISORDERS W/O MCC,DE,57.0,6899.31,5840.745,137.787576
064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W MCC,KY,43.0,11293.04,10192.765,222.920249


In [216]:
drg_by_states_min = sorted_df_by_drg_min.index.get_level_values('Provider State')

In [217]:
drg_min_states_counter = Counter(drg_by_states_min)
drg_min_states_dict = dict(drg_min_states_counter)
drg_min_states_df = pd.DataFrame(drg_min_states_dict.items())
drg_min_states_df.rename(columns={0: 'Provider State',1:'DRG_qty'}, inplace=True)
drg_min_states_df = drg_min_states_df.set_index('Provider State')

In [218]:
sorted_drg_min_states_df = drg_min_states_df.sort_values(by='DRG_qty')
sorted_drg_min_states_df.plot_bokeh.bar(show_figure=True, legend = 'top_left', title='States with cheapest DRG')

Figure(id='5204', ...)

To answer that question is necesary to clarify:

1.1. By **Avg Total Payments [$]**

💰 According to **Average Total Payments**  the three most expensive states✳️ are:

1. **California** with $164993988.  
2. **Texas** with $10967057.
3. **New York** with $108259026.

💰 According to **Average Total Payments**  the three most cheapest states are:

1. **West Virginia** with $2815426.
2. **Vermont** with $3176902.
3. **Arkansas** with $3366222.  
  

1.2. By **Discharges** (attended patient)

🤕 The three most expensive states are:
1. **Alaska** with $550.
2. **Hawaii** with $503.
3. **Wyoming** with $443.  


🤕 The three most cheapest states are:
1. **Delaware** with $215.
2. **Kentucky** with $226.
3. **Tennessee** with $227. 


2.3. By  **DRG** 

💉 The most expensive states by DRG are:
* **Alaska**: 42 DRG
* **Hawaii**: 21 DRG
* **Utah**: 13 DRG
* **Wyoming**: 8 DRG
* **West Virginia**: 7 DRG

💉 The most cheapest states by DRG are:
* **Delaware**: 20 DRG
* **Michigan**: 14 DRG
* **New Jersey**: 12 DRG
* **Maine**: 8 DRG
* **California**: 6 DRG


### 2. Cheapest Hospitals are in the same state?

 

In [219]:
df_by_hospital = df.groupby(['Provider State','Provider Name']).median()
df_by_hospital

Total Discharges  \
Provider State Provider Name                                          
AK             ALASKA NATIVE MEDICAL CENTER                    20.0   
               ALASKA REGIONAL HOSPITAL                        15.0   
               BARTLETT REGIONAL HOSPITAL                      13.0   
               CENTRAL PENINSULA GENERAL HOSPITAL              19.0   
               FAIRBANKS MEMORIAL HOSPITAL                     15.0   
...                                                             ...   
WY             MOUNTAIN VIEW REGIONAL HOSPITAL                 38.0   
               RIVERTON MEMORIAL HOSPITAL                      17.0   
               SHERIDAN MEMORIAL HOSPITAL                      19.0   
               ST JOHNS MEDICAL CENTER                         12.0   
               WYOMING MEDICAL CENTER                          19.0   

                                                   Avg Total Payments [$]  \
Provider State Provider Name                                                
AK             ALASKA NATIVE MEDICAL CENTER                      12160.37   
               ALASKA REGIONAL HOSPITAL                          10902.28   
               BARTLETT REGIONAL HOSPITAL                        10996.56   
               CENTRAL PENINSULA GENERAL HOSPITAL                 9917.46   
               FAIRBANKS MEMORIAL HOSPITAL                       14215.73   
...                                                                   ...   
WY             MOUNTAIN VIEW REGIONAL HOSPITAL                   11458.33   
               RIVERTON MEMORIAL HOSPITAL                         6340.50   
               SHERIDAN MEMORIAL HOSPITAL                         7978.81   
               ST JOHNS MEDICAL CENTER                           22607.10   
               WYOMING MEDICAL CENTER                             8994.55   

                                                   Avg Medicare Payments [$]  \
Provider State Provider Name                                                   
AK             ALASKA NATIVE MEDICAL CENTER                         11163.17   
               ALASKA REGIONAL HOSPITAL                              7250.90   
               BARTLETT REGIONAL HOSPITAL                           10091.01   
               CENTRAL PENINSULA GENERAL HOSPITAL                    8552.53   
               FAIRBANKS MEMORIAL HOSPITAL                          13079.86   
...                                                                      ...   
WY             MOUNTAIN VIEW REGIONAL HOSPITAL                      10435.72   
               RIVERTON MEMORIAL HOSPITAL                            5014.70   
               SHERIDAN MEMORIAL HOSPITAL                            7158.45   
               ST JOHNS MEDICAL CENTER                              21555.22   
               WYOMING MEDICAL CENTER                                7168.25   

                                                   Avg. Cost/attended patient [$]  
Provider State Provider Name                                                       
AK             ALASKA NATIVE MEDICAL CENTER                            575.913333  
               ALASKA REGIONAL HOSPITAL                                610.165897  
               BARTLETT REGIONAL HOSPITAL                              699.151538  
               CENTRAL PENINSULA GENERAL HOSPITAL                      589.173896  
               FAIRBANKS MEMORIAL HOSPITAL                             870.206923  
...                                                                           ...  
WY             MOUNTAIN VIEW REGIONAL HOSPITAL                         336.933467  
               RIVERTON MEMORIAL HOSPITAL                              399.894545  
               SHERIDAN MEMORIAL HOSPITAL                              380.314848  
               ST JOHNS MEDICAL CENTER                                 910.006667  
               WYOMING MEDICAL CENTER                                  420.6

In [220]:
df_by_hospital = df_by_hospital.sort_values(['Avg. Cost/attended patient [$]'], ascending=[1])
df_by_hospital.head(60)

Total Discharges  \
Provider State Provider Name                                                          
MA             ADCARE HOSPITAL OF WORCESTER INC                              1571.0   
TX             WESTBURY COMMUNITY HOSPITAL, LLC                               553.0   
OH             WOODS AT PARKSIDE,THE                                          431.0   
PA             VALLEY FORGE MEDICAL CENTER AND HOSPITAL                       154.0   
               EAGLEVILLE HOSPITAL                                            122.0   
GA             TURNING POINT HOSPITAL                                          91.0   
NC             WILMINGTON TREATMENT CENTER                                     99.0   
WI             MIDWEST ORTHOPEDIC SPECIALTY HOSPITAL, LLC                     293.0   
FL             FLORIDA HOSPITAL                                               189.5   
VA             CATAWBA HOSPITAL                                               120.0   
KS             KANSAS CITY ORTHOPAEDIC INSTITUTE                              260.0   
MI             HEALTHSOURCE SAGINAW                                            83.0   
KS             SALINA SURGICAL HOSPITAL                                       218.0   
PA             COORDINATED HEALTH ORTHOPEDIC HOSPITAL                         223.0   
OH             INSTITUTE FOR ORTHOPEDIC SURGERY                               232.0   
NE             LINCOLN SURGICAL HOSPITAL                                      212.0   
AZ             ARIZONA ORTHOPEDIC SURGICAL HOSPITAL                           204.0   
AR             ARKANSAS SURGICAL HOSPITAL                                     199.5   
CA             KERN MEDICAL CENTER                                            178.0   
TX             AUSTIN SURGICAL HOSPITAL                                       173.0   
CA             COLLEGE HOSPITAL COSTA MESA                                    131.5   
TX             METHODIST HOSPITAL                                             114.0   
MA             SOUTHCOAST HOSPITAL GROUP, INC                                  86.0   
OK             OKLAHOMA CENTER FOR ORTHOPAEDIC & MULTI-SP                     136.0   
CA             SUTTER SURGICAL HOSPITAL - NORTH VALLEY                        158.0   
PA             KENSINGTON HOSPITAL                                             49.0   
VA             EASTERN STATE HOSPITAL                                          57.0   
NJ             COMMUNITY MEDICAL CENTER                                        76.0   
TN             METHODIST HEALTHCARE MEMPHIS HOSPITALS                          90.0   
MI             WILLIAM BEAUMONT HOSPITAL                                       93.5   
FL             ST VINCENT'S MEDICAL CENTER                                     66.5   
DE             CHRISTIANA CARE HEALTH SERVICES, INC.                           98.0   
NC             MEMORIAL MISSION HOSPITAL AND ASHEVILLE SURGERY CE              71.5   
FL             NAPLES COMMUNITY HOSPITAL                                       71.5   
TX             BAPTIST MEDICAL CENTER                                          80.0   
WI             OAK LEAF SURGCL HSPTL                                          116.0   
NJ             VIRTUA WEST JERSEY HOSPITALS BERLIN                             68.0   
TX             MEMORIAL HERMANN HOSPITAL SYSTEM                                78.0   
AL             HUNTSVILLE HOSPITAL                                             76.0   
MT             GREAT FALLS CLINIC MEDICAL CENTER                              116.0   
FL             NORTH FLORIDA REGIONAL MEDICAL CENTER                           58.0   
NY             NEW YORK-PRESBYTERIAN HOSPITAL                                 127.0   
OH             SURGICAL HOSPITAL AT SOUTHWOODS                                 97.0   
TN             MEMORIAL HEALTHCARE SYSTEM, INC                                 64.5   
MI             WILLIAM BEAUMONT HOSPITAL-TROY                    

In [221]:
sorted_df_by_hospital = df_by_hospital.groupby(level=0).head(1)
sorted_df_by_hospital

,,Total Discharges,Avg Total Payments [$],Avg Medicare Payments [$],Avg. Cost/attended patient [$]
Provider State,Provider Name,,,,
MA,ADCARE HOSPITAL OF WORCESTER INC,1571.0,4967.560,4284.520,3.162037
TX,"WESTBURY COMMUNITY HOSPITAL, LLC",553.0,4637.260,3894.550,8.385642
OH,"WOODS AT PARKSIDE,THE",431.0,3913.140,3258.090,9.079211
PA,VALLEY FORGE MEDICAL CENTER AND HOSPITAL,154.0,4244.290,3597.980,27.560325
GA,TURNING POINT HOSPITAL,91.0,3291.310,2745.320,36.168242
NC,WILMINGTON TREATMENT CENTER,99.0,3639.470,2676.260,36.762323
WI,"MIDWEST ORTHOPEDIC SPECIALTY HOSPITAL, LLC",293.0,11790.350,9246.840,40.240102
FL,FLORIDA HOSPITAL,189.5,7595.750,5743.580,40.308414
VA,CATAWBA HOSPITAL,120.0,4990.100,4159.640,41.584167


In [222]:
df_by_expensive_hospital = df.groupby(['Provider State','Provider Name', 'DRG Definition']).median()
df_by_expensive_hospital.head(3)

Total Discharges  \
Provider State Provider Name                DRG Definition                                                         
AK             ALASKA NATIVE MEDICAL CENTER 065 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFAR...              18.0   
                                            066 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFAR...              20.0   
                                            069 - TRANSIENT ISCHEMIA                                        12.0   

                                                                                                Avg Total Payments [$]  \
Provider State Provider Name                DRG Definition                                                               
AK             ALASKA NATIVE MEDICAL CENTER 065 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFAR...                14650.83   
                                            066 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFAR...                10308.50   
                                            069 - TRANSIENT ISCHEMIA                                           9175.00   

                                                                                                Avg Medicare Payments [$]  \
Provider State Provider Name                DRG Definition                                                                  
AK             ALASKA NATIVE MEDICAL CENTER 065 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFAR...                   13597.72   
                                            066 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFAR...                    9469.10   
                                            069 - TRANSIENT ISCHEMIA                                              8425.66   

                                                                                                Avg. Cost/attended patient [$]  
Provider State Provider Name                DRG Definition                                                                      
AK             ALASKA NATIVE MEDICAL CENTER 065 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFAR...                      813.935000  
                                            066 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFAR...                      515.425000  
                                            069 - TRANSIENT ISCHEMIA                                                764.583333

In [223]:
df_by_expensive_hospital = df_by_expensive_hospital.sort_values(['Avg. Cost/attended patient [$]'], ascending=[0])
df_by_expensive_hospital.head(60)

Total Discharges  \
Provider State Provider Name                                      DRG Definition                                                         
CA             STANFORD HOSPITAL                                  207 - RESPIRATORY SYSTEM DIAGNOSIS W VENTILATOR...              11.0   
               WASHINGTON HOSPITAL                                207 - RESPIRATORY SYSTEM DIAGNOSIS W VENTILATOR...              11.0   
NY             WESTCHESTER MEDICAL CENTER                         870 - SEPTICEMIA OR SEVERE SEPSIS W MV 96+ HOURS                11.0   
IN             PINNACLE HOSPITAL                                  460 - SPINAL FUSION EXCEPT CERVICAL W/O MCC                     14.0   
NY             WESTCHESTER MEDICAL CENTER                         853 - INFECTIOUS & PARASITIC DISEASES W O.R. PR...              15.0   
CA             STANFORD HOSPITAL                                  870 - SEPTICEMIA OR SEVERE SEPSIS W MV 96+ HOURS                14.0   
TX             UNIVERSITY OF TEXAS MEDICAL BRANCH GAL             870 - SEPTICEMIA OR SEVERE SEPSIS W MV 96+ HOURS                12.0   
CA             UCSF MEDICAL CENTER                                207 - RESPIRATORY SYSTEM DIAGNOSIS W VENTILATOR...              12.0   
DC             HOWARD UNIVERSITY HOSPITAL                         853 - INFECTIOUS & PARASITIC DISEASES W O.R. PR...              14.0   
NY             JACOBI MEDICAL CENTER                              853 - INFECTIOUS & PARASITIC DISEASES W O.R. PR...              11.0   
MD             JOHNS HOPKINS HOSPITAL, THE                        853 - INFECTIOUS & PARASITIC DISEASES W O.R. PR...              14.0   
NY             WESTCHESTER MEDICAL CENTER                         329 - MAJOR SMALL & LARGE BOWEL PROCEDURES W MCC                15.0   
PA             MILTON S HERSHEY MEDICAL CENTER                    469 - MAJOR JOINT REPLACEMENT OR REATTACHMENT O...              15.0   
NY             WESTCHESTER MEDICAL CENTER                         207 - RESPIRATORY SYSTEM DIAGNOSIS W VENTILATOR...              15.0   
WA             UNIVERSITY OF WASHINGTON MEDICAL CTR               853 - INFECTIOUS & PARASITIC DISEASES W O.R. PR...              12.0   
CA             COMMUNITY HOSPITAL OF THE MONTEREY PENINSULA       870 - SEPTICEMIA OR SEVERE SEPSIS W MV 96+ HOURS                13.0   
MN             HENNEPIN COUNTY MEDICAL CENTER                     853 - INFECTIOUS & PARASITIC DISEASES W O.R. PR...              13.0   
CA             JOHN MUIR MEDICAL CENTER - WALNUT CREEK CAMPUS     870 - SEPTICEMIA OR SEVERE SEPSIS W MV 96+ HOURS                13.0   
NY             ST BARNABAS HOSPITAL                               207 - RESPIRATORY SYSTEM DIAGNOSIS W VENTILATOR...              11.0   
CA             POMERADO HOSPITAL                                  853 - INFECTIOUS & PARASITIC DISEASES W O.R. PR...              11.0   
NY             BRONX-LEBANON HOSPITAL CENTER                      853 - INFECTIOUS & PARASITIC DISEASES W O.R. PR...              12.0   
IL             ADVOCATE ILLINOIS MASONIC MEDICAL CENTER           207 - RESPIRATORY SYSTEM DIAGNOSIS W VENTILATOR...              14.0   
CA             SIMI VALLEY HOSPITAL & HEALTH CARE SERVICES        329 - MAJOR SMALL & LARGE BOWEL PROCEDURES W MCC                11.0   
PA             ALBERT EINSTEIN MEDICAL CENTER                     329 - MAJOR SMALL & LARGE BOWEL PROCEDURES W MCC                13.0   
MD             NORTHWEST HOSPITAL CENTER                          460 - SPINAL FUSION EXCEPT CERVICAL W/O MCC                     11.0   
AK             FAIRBANKS MEMORIAL HOSPITAL                        329 - MAJOR SMALL & LARGE BOWEL PROCEDURES W MCC                13.0   
TX             UT SOUTHWESTERN UNIVERSITY HOSPITAL                870 - SEPTICEMIA OR SEVERE SEPSIS W MV 96+ HOURS                13.0   
WA             UNIVERSITY OF WASHINGTON MEDICAL CTR               870 - SEPTICEMIA OR SEVERE SEPSIS W MV 96+ HOURS  